In [11]:
'''
DATA PREPROCESSING/EXPLORATION STEP
'''

'''
Implement data augmentation on Waldo images
'''
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.datasets import load_files
from glob import glob
import numpy as np
import cv2


datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255, # rescale the image before feeding it to the cnn
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

waldo_filenames = np.array(glob("waldo_dataset/original_waldo/*"))

In [12]:
'''
Generate augmented images for waldo (do it only once)
'''

# for filename in waldo_filenames:
#     img = load_img(filename)
#     x = img_to_array(img)
#     x = x.reshape((1,) + x.shape)
#     i = 0
#     for batch in datagen.flow(x, batch_size=1,
#                               save_to_dir='waldo_dataset/train/waldo', 
#                               save_prefix='new_waldo', 
#                               save_format='jpg'):
#         i += 1
#         if i > 20:
#             break  # the flow may yield batches indefinitely

'\nGenerate augmented images for waldo (do it only once)\n'

In [13]:
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    return files, targets

all_files, all_targets = load_dataset('waldo_dataset/train_32')
test_files, test_targets = load_dataset('waldo_dataset/test')

'''
Shuffle the data to avoid a bias
'''
rand = np.random.RandomState(10)
shuffle = rand.permutation(len(all_files))
all_files, all_targets = all_files[shuffle], all_targets[shuffle]

'''
Show statistics about the data
'''
waldoFiles = np.array(glob("waldo_dataset/train_32/waldo/*"))
notWaldoFiles = np.array(glob("waldo_dataset/train_32/notwaldo/*"))
print('Waldo dataset contains ' + str(len(waldoFiles)) + ' images')
print('Notwaldo dataset contains ' + str(len(notWaldoFiles)) + ' images')

Waldo dataset contains 609 images
Notwaldo dataset contains 5337 images


In [14]:
def resize_img(filename, key):
    image = cv2.imread(filename)
    r = 100.0 / image.shape[1]
    dim = (100, int(image.shape[0] *r))
    imageresized = cv2.resize(image,(32,32),dim,interpolation = cv2.INTER_AREA)
    cv2.imwrite('imageresized_{}.jpg'.format(key), imageresized)
    
# resize_img('test.jpg', 'asdf')
# waldoFilesnew = np.array(glob("waldo_dataset/test/waldo/*"))
# key = 0    
# for image_file in waldoFilesnew:
#     resize_img(image_file, key)
#     key += 1

In [ ]:
'''
CREATE THE SVM CLASSIFIER
'''

In [ ]:
'''
TEST THE SVM CLASSIFIER
'''

In [15]:
'''
CREATE THE CNN CLASSIFIER
'''
'''
Preprocess the images in order to use Keras with tensorflow backend
(I took this code from the dog breed classification project)
'''

from keras.preprocessing import image                  

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(32, 32))
    # convert PIL.Image.Image type to 3D tensor with shape (64, 64, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 64, 64, 3) and return 4D tensor
    result = np.expand_dims(x, axis=0)
    return result

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [16]:
from tqdm import tqdm
train_tensors = paths_to_tensor(all_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255


100%|██████████| 20/20 [00:00<00:00, 3277.57it/s]

In [17]:
""" Create the model """
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

### TODO: Define your architecture.
model.add(Conv2D(filters=16, kernel_size=2, strides=1, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
__________

In [27]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
'''
Train the model
'''
model.fit(train_tensors, all_targets, 
          validation_split=0.2,
          epochs=3, batch_size=20, 
#           callbacks=[checkpointer], 
          verbose=1)

Train on 4756 samples, validate on 1190 samples
Epoch 1/3
4756/4756 [==============================] - 2s - loss: 0.0758 - acc: 0.9708 - val_loss: 0.0719 - val_acc: 0.9706
Epoch 2/3
4756/4756 [==============================] - 2s - loss: 0.0715 - acc: 0.9727 - val_loss: 0.0528 - val_acc: 0.9790
Epoch 3/3
4756/4756 [==============================] - 2s - loss: 0.0609 - acc: 0.9786 - val_loss: 0.0519 - val_acc: 0.9773


In [40]:
'''
TEST THE CNN CLASSIFIER
'''
def img_to_tensor(img):
#     x = img.reshape((1,) + img.shape)
#     print(x.shape)
    # convert 3D tensor to 4D tensor with shape (1, 64, 64, 3) and return 4D tensor
    result = np.expand_dims(img, axis=0)
    print(result.shape)
    return result

def predict_waldo(img):
#     img = load_img(img_path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
#     print(np.squeeze(model.predict(x)))
    return np.squeeze(model.predict(x))

count_correct = 0
threshold = 0.6
for i in range(len(test_files)):
    file_path = test_files[i]
    file = load_img(file_path, target_size=(32, 32))
    file = image.img_to_array(file).astype('float32')/255
    score = predict_waldo(file)
    if ((score < threshold and test_targets[i] == 0) or (score >= threshold and test_targets[i] == 1)):
        count_correct += 1
        
print('the accuracy score of the CNN classifier is ', count_correct / len(test_files))

# file = load_img('test.jpg', target_size=(32, 32))
# file = image.img_to_array(file).astype('float32')/255
# predict_waldo(file)


the accuracy score of the CNN classifier is  0.9


In [11]:
'''
OBJECT DETECTION PART
'''

# Create a sliding window
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
            
def pyramid(image, scale, minSize=(30, 30)):
    # yield the original image
    yield(image)
 
    # keep looping over the pyramid
    while True:
        # compute the new dimensions of the image and resize it
        w = int(image.shape[1] / scale)
        image = resize_im(image, width=w)
 
        # if the resized image does not meet the supplied minimum
        # size, then stop constructing the pyramid
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
 
        # yield the next image in the pyramid
        yield(image)

In [12]:
''' Save image from a window '''
img = image.load_img('test.jpg', target_size=(32, 32))
img = image.img_to_array(img).astype('float32')/255
def save_img(img, key):    
#     cv2.imwrite('waldo_dataset/hard_neg/hardmin_{}.jpg'.format(key), cv2.cvtColor(img.astype('float32')*255, cv2.COLOR_RGB2BGR))
    cv2.imwrite('waldo_dataset/hard_neg/hardmin_{}.jpg'.format(key), img.astype('float32')*255)

# save_img(img)

In [ ]:
'''
TEST THE OBJECT DETECTION PART
'''
originalFiles = np.array(glob("waldo_dataset/original_images/*"))
threshold = 0.5

from time import sleep
image = cv2.imread(originalFiles[9])
winW = 32
winH = 32
y_pred = []
key = 0
for resized in pyramid(image, scale=0.5):
    for (x, y, window) in sliding_window(resized, stepSize=16, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            window = window.astype('float32')/255
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            if predict_waldo(window) > threshold:
                cv2.rectangle(image, (x, y), (x + winW, y + winH), (255, 0, 0), 2)
                save_img(window, key)
                key += 1
                
            # since we do not have a classifier, we'll just draw the window
            clone = resized.copy()
            cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
            cv2.imshow("Window", clone)
            cv2.waitKey(33)
            sleep(0.025)

0.5699414610862732
0.13005377352237701
0.028000127524137497
0.03821966052055359
0.007516391109675169
0.02852826751768589
0.048245713114738464
0.03613189607858658
0.011206919327378273
0.9651600122451782
0.06280162185430527
0.3647853434085846
0.020308081060647964
0.05694667622447014
0.03352495655417442
0.0329289510846138
0.1475488543510437
0.04119669273495674
0.03745311498641968
0.1259559541940689
0.1951843798160553
0.04207998141646385
0.09010880440473557
0.18906177580356598
0.45715829730033875
0.9893320798873901
0.9072889089584351
0.12945254147052765
0.8085305690765381
0.1302148699760437
0.12213896960020065
0.0055526020005345345
0.001196898752823472
0.012072390876710415
0.05866742506623268
0.002260425826534629
0.0092220613732934
0.024180395528674126
0.0492103211581707
0.38039934635162354
0.0016423184424638748
0.021523354575037956
0.0899374857544899
0.7586046457290649
0.005731341429054737
0.04209489747881889
0.0016117340419441462
0.09770411252975464
0.12848083674907684
0.2738544344902038

0.03224577754735947
0.14324814081192017
0.21771620213985443
0.49992620944976807
0.5496153235435486
0.6607288122177124
0.00032389225088991225
0.4172457754611969
0.5195401310920715
0.14122092723846436
0.9423377513885498
0.0673426017165184
0.7127451300621033
0.5921359062194824
0.4611453711986542
0.5974372625350952
0.1916302889585495
0.017586074769496918
0.002868084469810128
0.25750765204429626
0.2987655699253082
0.0528838112950325
0.27649441361427307
0.07309176027774811
0.042605508118867874
0.03776567429304123
0.1564732789993286
0.023882756009697914
0.09337656944990158
0.005237502511590719
0.2091667205095291
0.004170767031610012
0.19981645047664642
0.08535505086183548
0.07040417939424515
0.09716267883777618
0.06307045370340347
0.12779520452022552
0.010358634404838085
0.00382242607884109
0.05705671012401581
0.02719436213374138
0.010825824923813343
0.014980542473495007
0.07675018906593323
0.06148768216371536
0.013586794957518578
0.03130272775888443
0.06518395245075226
0.008859941735863686
0

0.009202615357935429
0.03798438981175423
0.0028059938922524452
0.040686216205358505
0.0031066741794347763
0.006918188650161028
0.016429241746664047
0.009408815763890743
0.004051052499562502
0.001016847905702889
0.03641756251454353
0.03250005468726158
0.2214202731847763
0.010193806141614914
0.008431128226220608
0.027539828792214394
0.7890257239341736
0.08580513298511505
0.18257619440555573
0.041707515716552734
0.0436575710773468
0.0025591005105525255
0.01793530210852623
0.06531203538179398
0.3802637755870819
0.32130834460258484
0.07442766427993774
0.14214473962783813
0.03132103756070137
0.0004552312893792987
0.005236429162323475
0.00394471175968647
0.014556879177689552
0.05489550158381462
0.04739265888929367
0.004085300490260124
0.0013768700882792473
0.0018592372070997953
0.001571019645780325
0.0007800846360623837
0.0795227438211441
0.11180338263511658
0.21911263465881348
0.366763710975647
0.03205978125333786
0.1362799108028412
0.5654830932617188
0.03999725356698036
0.06969485431909561


0.6346556544303894
0.7609084844589233
0.01578131690621376
0.08785056322813034
0.9609317183494568
0.5614158511161804
0.07759504020214081
0.003611912950873375
0.010159429162740707
0.0017619752325117588
0.003609701758250594
0.05749150738120079
0.04271845519542694
0.15524226427078247
0.020779967308044434
0.011017617769539356
0.03227753937244415
0.07127396762371063
0.07032490521669388
0.09855291247367859
0.015497613698244095
0.21923953294754028
0.6896468997001648
0.033833060413599014
0.0017193173989653587
0.0010233836947008967
0.0005689263925887644
0.0020366644021123648
0.04467878118157387
0.06132074445486069
0.03183901309967041
0.07892876118421555
0.22043612599372864
0.6889004111289978
0.03855842351913452
0.013520905748009682
0.017835762351751328
0.05857959762215614
0.0010474969167262316
0.08913359045982361
0.707297682762146
0.04474844038486481
0.01250968687236309
0.006346552167087793
0.016812235116958618
0.7523056864738464
0.00026254254044033587
0.015817657113075256
0.020627306774258614
0

0.1542392075061798
0.12809249758720398
0.3176189363002777
0.3334892690181732
0.03974223509430885
0.04177228733897209
0.04676152393221855
0.15797212719917297
0.048600777983665466
0.050833843648433685
0.0006075481651350856
0.200526624917984
0.24449381232261658
0.0021063906606286764
0.004305501934140921
0.006109899841248989
0.019000234082341194
0.0011051360052078962
0.1746799349784851
0.022653432562947273
0.0479922890663147
0.013254577293992043
0.005628823768347502
0.0039369272999465466
0.23684322834014893
0.1422676146030426
0.00021963630570098758
0.005258145276457071
0.03813378885388374
0.02098885551095009
0.011499294079840183
0.6702277064323425
0.008213499560952187
0.44907069206237793
0.1904900074005127
0.03230945020914078
0.4283923804759979
0.7120980620384216
0.1877952665090561
0.012703263200819492
0.0898359939455986
0.06710659712553024
0.1276058852672577
0.10805324465036392
0.14259347319602966
0.045648928731679916
0.05593007802963257
0.805469274520874
0.005050124600529671
0.2856873273

...

0.0030420550610870123
0.323006272315979
0.07303369045257568
0.009454469196498394
0.40364503860473633
0.2540963590145111
0.4937799274921417
0.37750691175460815
0.211748406291008
0.6618937253952026
0.1768418252468109
0.02696644514799118
0.44400453567504883
0.2947665750980377
0.04081839695572853
0.3139696419239044
0.01959134452044964
0.04829091578722
0.09385529160499573
0.023144131526350975
0.021329713985323906
0.034484412521123886
0.005021471530199051
0.08042705059051514
0.010543184354901314
0.020769886672496796
0.846064031124115
0.8972815275192261
0.00420943321660161
0.3216450810432434
0.6448330283164978
0.12270091474056244
0.024512561038136482
0.10791712254285812
0.010809721425175667
0.33829784393310547
0.0016854864079505205
0.03861912712454796
0.039914824068546295
0.14529508352279663
0.009093966335058212
0.08167026191949844
0.26159775257110596
0.016864659264683723
0.09870077669620514
0.00429975101724267
0.0014953598147258162
0.01149742491543293
0.019791167229413986
0.06638595461845398

0.005592000670731068
0.19849035143852234
0.18042880296707153
0.3922116160392761
0.15324249863624573
0.17181770503520966
0.09038016200065613
0.2631753385066986
0.05112443491816521
0.012927364557981491
0.02107909880578518
0.088506780564785
0.06070324778556824
0.10915438830852509
0.010159232653677464
0.7458639740943909
0.2810050845146179
0.043864473700523376
0.3402617871761322
0.1312236189842224
0.0035373112186789513
0.0799669474363327
0.012952902354300022
0.02092527039349079
0.03759301081299782
0.13660560548305511
0.0502612330019474
0.11367461085319519
0.021294109523296356
0.6368899941444397
0.005602874327450991
0.3769940435886383
0.2647375762462616
0.7549184560775757
0.13830453157424927
0.09061643481254578
0.3516746461391449
0.11210384964942932
0.5082492828369141
0.423287570476532
0.041681405156850815
0.5248575210571289
0.011000354774296284
0.4364061951637268
0.07491675019264221
0.17080388963222504
0.011746199801564217
0.06400525569915771
0.2827530801296234
0.17736198008060455
0.3980163

0.1640274077653885
0.0736784040927887
0.07695019990205765
0.009489535354077816
0.2916669547557831
0.4209710359573364
0.5016427040100098
0.02323439158499241
0.029002012684941292
0.1666872203350067
0.002525912132114172
0.0180590208619833
0.09180343151092529
0.20530153810977936
0.01298055425286293
0.0059661115519702435
0.013449256308376789
0.040640853345394135
0.008839313872158527
0.051614005118608475
0.43553999066352844
0.2348054051399231
0.31082773208618164
0.5766937732696533
0.286953866481781
0.34128159284591675
0.1646794229745865
0.05408502370119095
0.18128381669521332
0.1065702885389328
0.5430318117141724
0.1978859007358551
0.004169054329395294
0.05525728315114975
0.15647806227207184
0.03931954875588417
0.09607794880867004
0.1974538266658783
0.14744088053703308
0.031797297298908234
0.04962878301739693
0.0052343010902404785
0.0620097815990448
0.24202093482017517
0.027469748631119728
0.03895585983991623
0.03241673856973648
0.10947675257921219
0.006554452236741781
0.007516789250075817
0

0.8315715789794922
0.6352047324180603
0.577013373374939
0.1495296061038971
0.46178707480430603
0.217011496424675
0.6269701719284058
0.14167219400405884
0.12830021977424622
0.17071682214736938
0.02760601043701172
0.3352002799510956
0.5081303715705872
0.5426254272460938
0.01715686172246933
0.3981071412563324
0.025482406839728355
0.010552052408456802
0.0015915720723569393
0.039928145706653595
0.04386259987950325
0.0457565113902092
0.08327876776456833
0.027166074141860008
0.051336418837308884
0.04313301295042038
0.10435616970062256
0.1000765785574913
0.13176757097244263
0.2953048646450043
0.035745225846767426
0.07956832647323608
0.042109571397304535
0.0822259932756424
0.574950098991394
0.05993138253688812
0.0025736303068697453
0.0016534948954358697
0.10846037417650223
0.02071530558168888
0.032911255955696106
0.7565879821777344
0.021261228248476982
0.02118103951215744
0.0029846758116036654
0.05331757292151451
0.11034432798624039
0.24962368607521057
0.03154480457305908
0.019269682466983795
0

0.01530824601650238
0.007937010377645493
0.011270275339484215
0.0007808476802892983
0.0020499348174780607
0.016681190580129623
0.0005984782474115491
0.01699654757976532
0.03789050504565239
0.0275674257427454
0.7271442413330078
0.47794419527053833
0.7227253317832947
0.004965431522578001
0.04188082367181778
0.12519529461860657
0.04789716750383377
0.15447521209716797
0.08454161882400513
0.019756104797124863
0.0330793671309948
0.041694436222314835
0.220267191529274
0.04309596121311188
0.3229725956916809
0.019459795206785202
0.005502610933035612
0.0290096215903759
0.02193554863333702
0.021517792716622353
0.5578652024269104
0.013668215833604336
0.004371062386780977
0.02875709906220436
0.16491319239139557
0.14154905080795288
0.22458390891551971
0.009298106655478477
0.02646930143237114
0.07623954862356186
0.12909159064292908
0.1504921019077301
0.07567229121923447
0.02562584914267063
0.05855431780219078
0.007038375828415155
0.01221142802387476
0.07684725522994995
0.02779831551015377
0.075469657